In [3]:
print("Helllo Jupyter")

Helllo Jupyter


In [5]:
!pip install tensorflow

     ------------------------------------ 276.5/276.5 MB 967.1 kB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 KB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 1.7 MB/s eta 0:00:00
     -------------------------------------- 440.8/440.8 KB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 KB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 4.3/4.3 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 1.4 MB/s eta 0:00:00
     -------------------------------------- 430.5/430.5 KB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 14.9/14.9 MB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 24.4/24.4 MB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 65.5

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
filelock 3.12.3 requires typing-extensions>=4.7.1; python_version < "3.11", but you have typing-extensions 4.5.0 which is incompatible.
You should consider upgrading via the 'C:\Users\HP\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
import tensorflow as tf
import os
from os.path import isfile, join
import numpy as np
import shutil
from tensorflow import keras
from pathlib import Path
from IPython.display import display, Audio
import subprocess

In [18]:
data_directory = "./datasets"
audio_folder = "audio"
noise_folder = "_background_noise_"

audio_path = os.path.join(data_directory, audio_folder)
noise_path = os.path.join(data_directory, noise_folder)

In [19]:
audio_path

'./datasets\\audio'

In [20]:
valid_split = 0.1

shuffle_seed = 43

sample_rate = 16000

scale = 0.5

batch_size = 128

epochs = 15

In [21]:
for folder in os.listdir(data_directory):
    if os.path.isdir(os.path.join(data_directory, folder)):
        if folder in [audio_folder, noise_folder]:
            
            continue
        elif folder in ["other", "_background_noise_"]:
            
            shutil.move(
                os.path.join(data_directory, folder),
                os.path.join(noise_path, folder),
            )
        else:
            shutil.move(
                os.path.join(data_directory, folder),
                os.path.join(audio_path, folder),
            )

In [22]:
noise_paths = []
for subdir in os.listdir(noise_path):
    subdir_path = Path(noise_path) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

In [23]:
noise_paths

['datasets\\_background_noise_\\other\\exercise_bike.wav',
 'datasets\\_background_noise_\\other\\pink_noise.wav']

In [24]:
command = (
    "for dir in `ls -1 " + noise_path + "`; do "
    "for file in `ls -1 " + noise_path + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)

In [25]:
os.system(command)
def load_noise_sample(path):
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels=1
    )
    if sampling_rate == sample_rate:
        slices = int(sample.shape[0] / sample_rate)
        sample = tf.split(sample[: slices * sample_rate], slices)
        return sample
    else:
        print("Sampling rate for",path, "is incorrect")
        return None


noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)

Sampling rate for datasets\_background_noise_\other\exercise_bike.wav is incorrect
Sampling rate for datasets\_background_noise_\other\pink_noise.wav is incorrect
